In [29]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import preprocess_input



In [30]:
def read_image(file_path, target_size=(128, 128)):
    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    return image


In [31]:
def encode_image(encoder, image):
    image = np.expand_dims(image, axis=0)
    encoded_image = encoder.predict(preprocess_input(image))
    return encoded_image


In [32]:
def find_most_similar_product(encoder, test_image, training_images):
    test_encoding = encode_image(encoder, test_image)

    # Compare the test image encoding with each training image encoding
    similarities = [np.sum(np.square(test_encoding - encode_image(encoder, train_image))) for train_image in
                    training_images]

    # Find the index of the most similar training image
    most_similar_index = np.argmin(similarities)
    return most_similar_index


In [33]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras import backend, layers, metrics
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
import tensorflow as tf


def get_encoder(input_shape, l2_regularization=0.01):
    """ Returns the image encoding model using MobileNet """
    pretrained_model = MobileNet(
        input_shape=input_shape,
        weights='imagenet',
        include_top=False,
        pooling='avg',
    )
    
    for layer in pretrained_model.layers[:-27]:
        layer.trainable = False

    encode_model = Sequential([
        pretrained_model,
        layers.Flatten(),
        layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(l2_regularization)),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(l2_regularization)),
        layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    ], name="Encode_Model")
    
    return encode_model
# Load the saved encoder
def load_encoder(input_shape=(128, 128, 3)):
    encoder = get_encoder(input_shape)
    encoder.load_weights("encoder")
    return encoder

In [42]:


encoder =load_encoder()

# Load training images for the first case
training_images_dir = "D:\semester 7\projectcv\Data\Product Recoginition\Training Data"
training_images = [read_image(os.path.join(training_images_dir, str(i), f"web1.png")) for i in range(1, 41)]

# Path to the directory containing test images for the first case
test_case1_dir = "D:\semester 7\Data_test"

true_res=0
false_res=0
# Case 1: Testing on images of different products

for unseen_test_images in os.listdir(test_case1_dir):
     unseen_test_image_path = os.path.join(test_case1_dir, f"{unseen_test_images}")
    # unseen_test_image = read_image(unseen_test_image_path)
    
     unseen_test_image = read_image(unseen_test_image_path)


     unseen_most_similar = find_most_similar_product(encoder, unseen_test_image, training_images)
     print(f"Unseen Test Image: {unseen_test_images}, Most Similar Product: {unseen_most_similar+1}")
     unseen_test_image_name = os.path.splitext(unseen_test_images)[0]
     if(int(unseen_test_image_name)==unseen_most_similar+1):
          true_res+=1
     else:
          false_res+=1    

acc= (true_res/(true_res+false_res))*100
print(f"Accuracy : {acc}%")

1/1 [==============================] - 0s 40ms/step
Unseen Test Image: 1.png, Most Similar Product: 1
1/1 [==============================] - 0s 38ms/step
Unseen Test Image: 2.png, Most Similar Product: 2
1/1 [==============================] - 0s 33ms/step
Unseen Test Image: 3.png, Most Similar Product: 3
1/1 [==============================] - 0s 50ms/step
Unseen Test Image: 4.png, Most Similar Product: 4
1/1 [==============================] - 0s 43ms/step
Unseen Test Image: 5.png, Most Similar Product: 4
Accuracy : 80.0%
